In [1]:
import requests as req
from bs4 import BeautifulSoup as bs
import pandas as pd
import json
import selenium
import os
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
import time 
import datetime
import src.cleaning_functions as clf
#import the libraries that we need

In [2]:
#driver configuration
opciones=Options()

opciones.add_experimental_option('excludeSwitches', ['enable-automation'])
opciones.add_experimental_option('useAutomationExtension', False)
opciones.headless=False    # si True, no aperece la ventana (headless=no visible)
opciones.add_argument('--start-maximized')         # comienza maximizado
opciones.add_argument('--incognito')
opciones.add_argument("--no-sandbox")
opciones.add_argument("--disable-setuid-sandbox")

In [3]:
driver = "./chromedriver.exe"
driver = webdriver.Chrome(driver,options = opciones)
#driver creation with selected options

<ipython-input-3-11b2f6732cd6>:2: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(driver,options = opciones)


In [4]:
base_url ="https://www.reuters.com/news/archive/goldMktRpt?view=page&page="
end_url = "&pageSize=10"
all_url =[]
for i in range (12,223):
    all_url.append(base_url + str(i) + end_url)
all_url

#List of all URL to scrap
    

['https://www.reuters.com/news/archive/goldMktRpt?view=page&page=12&pageSize=10',
 'https://www.reuters.com/news/archive/goldMktRpt?view=page&page=13&pageSize=10',
 'https://www.reuters.com/news/archive/goldMktRpt?view=page&page=14&pageSize=10',
 'https://www.reuters.com/news/archive/goldMktRpt?view=page&page=15&pageSize=10',
 'https://www.reuters.com/news/archive/goldMktRpt?view=page&page=16&pageSize=10',
 'https://www.reuters.com/news/archive/goldMktRpt?view=page&page=17&pageSize=10',
 'https://www.reuters.com/news/archive/goldMktRpt?view=page&page=18&pageSize=10',
 'https://www.reuters.com/news/archive/goldMktRpt?view=page&page=19&pageSize=10',
 'https://www.reuters.com/news/archive/goldMktRpt?view=page&page=20&pageSize=10',
 'https://www.reuters.com/news/archive/goldMktRpt?view=page&page=21&pageSize=10',
 'https://www.reuters.com/news/archive/goldMktRpt?view=page&page=22&pageSize=10',
 'https://www.reuters.com/news/archive/goldMktRpt?view=page&page=23&pageSize=10',
 'https://www.re

In [5]:

url = 'https://www.reuters.com/news/archive/goldMktRpt?view=page&page=11&pageSize=10'
driver.get(url)
driver.implicitly_wait(3)

cookie=driver.find_element(By.CSS_SELECTOR,"#onetrust-pc-sdk > div.ot-pc-footer > div.ot-btn-container > button.save-preference-btn-handler.onetrust-close-btn-handler")
cookie.click()

#The first url to scrap. This one is alone (not in the loop) because we need it to also click the cookie botton

In [6]:

html=req.get(url).content
soup=bs(html, 'html.parser')

articulos = soup.find_all("article", class_="story")
link=articulos[0].find_all('a')
link
#create our soup to get the links

[<a href="/article/global-precious/precious-gold-steadies-above-1800-oz-as-pandemic-risks-linger-idUSL4N2TK1QZ">
 <img alt="" border="0" class="placeholder-image" src="https://s1.reutersmedia.net/resources_v2/images/1x1.png"/>
 </a>,
 <a href="/article/global-precious/precious-gold-steadies-above-1800-oz-as-pandemic-risks-linger-idUSL4N2TK1QZ">
 <h3 class="story-title">
 								PRECIOUS-Gold steadies above $1,800/oz as pandemic risks linger</h3>
 </a>]

In [7]:
listica = []
for a in range(0,10):
    listica.append(articulos[a].find_all('a'))
#obtain all the info of box "a" inside articles

In [8]:
todos_links = []
for b in range(0,10):
    todos_links.append(listica[b][0].attrs["href"])
#get all the links from listica

In [9]:
todos_titulos = []
for b in range(0,10):
    todos_titulos.append(listica[b][1].text.strip().split('\n')[0])
#get all the titles from listica

In [10]:
date = soup.find_all("article", class_="story")
hora=date[0].find_all('time')
#hora[0].text.strip().split('\n')[0]
#repeat the proccess but with the date

In [11]:
tiempito = []
for a in range(0,10):
    tiempito.append(articulos[a].find_all('time'))
#like listica, we take all the info from the box "time"

In [12]:
todos_date = []
for i in range(0,10):
    todos_date.append(tiempito[i][0].text.strip())
#like in links and names, we take the relevant info which are the dates

In [13]:

df = pd.DataFrame(list(zip(todos_date, todos_titulos,todos_links)),
               columns =['date', 'titles','url'])
#we zip the three lists into one pd dataframe

In [14]:
df

,date,titles,url
0,Jan 04 2022,"PRECIOUS-Gold steadies above $1,800/oz as pand...",/article/global-precious/precious-gold-steadie...
1,Jan 04 2022,PRECIOUS-Gold inches up after biggest sell-off...,/article/global-precious/precious-gold-inches-...
2,Jan 03 2022,PRECIOUS-Gold gains as preventive curbs increa...,/article/global-precious/precious-gold-gains-a...
3,Jan 03 2022,PRECIOUS-Gold falls over 1% as risk appetite g...,/article/global-precious/precious-gold-falls-o...
4,Jan 03 2022,PRECIOUS-Gold slips from six-week high on equi...,/article/global-precious/precious-gold-slips-f...
5,Jan 03 2022,PRECIOUS-Gold steady near 6-week high as Omicr...,/article/global-precious/precious-gold-steady-...
6,Jan 02 2022,PRECIOUS-Gold steady as rising bond yields cou...,/article/global-precious/precious-gold-steady-...
7,Dec 31 2021,PRECIOUS-Gold marks yearly dip as central bank...,/article/global-precious/precious-gold-marks-y...
8,Dec 31 2021,PRECIOUS-Lacklustre safe-haven demand tips gol...,/article/global-precious/precious-lacklustre-s...
9,Dec 31 2021,PRECIOUS-Lacklustre safe-haven demand tips gol...,/article/global-precious/precious-lacklustre-s...


In [15]:
for u in all_url:
    driver.get(u)
    driver.implicitly_wait(3)
    
    html=req.get(u).content
    soup=bs(html, 'html.parser')

    articulos = soup.find_all("article", class_="story")
    link=articulos[0].find_all('a')
    
    listica = []
    for a in range(0,10):
        listica.append(articulos[a].find_all('a'))
        
    todos_links = []
    for b in range(0,10):
        todos_links.append(listica[b][0].attrs["href"])
        
    todos_titulos = []
    for b in range(0,10):
        todos_titulos.append(listica[b][1].text.strip().split('\n')[0])
        
    
    hora=articulos[0].find_all('time')
    tiempito = []
    for a in range(0,10):
        tiempito.append(articulos[a].find_all('time'))
        
    todos_date = []
    for i in range(0,10):
        todos_date.append(tiempito[i][0].text.strip())
    
    df_base = pd.DataFrame(list(zip(todos_date, todos_titulos,todos_links)),
               columns =['date', 'titles','url'])
    
    df=pd.concat([df,df_base],ignore_index=True)
    
   #big loop in order to get it at onces 

In [16]:
df
#check the database

,date,titles,url
0,Jan 04 2022,"PRECIOUS-Gold steadies above $1,800/oz as pand...",/article/global-precious/precious-gold-steadie...
1,Jan 04 2022,PRECIOUS-Gold inches up after biggest sell-off...,/article/global-precious/precious-gold-inches-...
2,Jan 03 2022,PRECIOUS-Gold gains as preventive curbs increa...,/article/global-precious/precious-gold-gains-a...
3,Jan 03 2022,PRECIOUS-Gold falls over 1% as risk appetite g...,/article/global-precious/precious-gold-falls-o...
4,Jan 03 2022,PRECIOUS-Gold slips from six-week high on equi...,/article/global-precious/precious-gold-slips-f...
...,...,...,...
2115,Jan 02 2020,"PRECIOUS-Gold edges up on weaker dollar, set f...",/article/global-precious/precious-gold-edges-u...
2116,Jan 02 2020,PRECIOUS-Gold races to 3-month peak on doubts ...,/article/global-precious/precious-gold-races-t...
2117,Jan 02 2020,"PRECIOUS-Gold rises back towards 3-month high,...",/article/global-precious/precious-gold-rises-b...
2118,Jan 02 2020,PRECIOUS-Gold prices inch higher on U.S. dolla...,/article/global-precious/precious-gold-prices-...


In [19]:
df.to_csv('news_base2.csv',encoding='utf-8-sig')
#export the data base to a csv

In [20]:
df['date'] = pd.to_datetime(df['date'],format='%b %d %Y')
#format the time

In [22]:

df['year'] = pd.DatetimeIndex(df['date']).year
df['month'] = pd.DatetimeIndex(df['date']).month
df['day'] = pd.DatetimeIndex(df['date']).day
#create the colims for the year,month and day

In [24]:
df["titles"] = df.titles.apply(clf.clean_titles)
#clean titles

In [27]:
df["url"] = df.url.apply(clf.clean_url)
#apply function to add the url

In [29]:
df = df.drop(["date"],axis=1)
#drop the date column

In [31]:
df.drop(df[df.year > 2021].index, inplace=True)

In [33]:
df.drop(df[df.year < 2020].index, inplace=True)

In [36]:
df = df.iloc[:, [2,3,4,0,1]]
#organize the df columns in the order we want

In [39]:
df.to_csv('news_clean.csv',encoding='utf-8-sig',index=False)